In [ ]:
#py_wrf_arps version : 2308f94f39bfcb528a6831fa73111bec80e03a5a , the current date is :  2025-02-21 13:59:19.987486
from py_wrf_arps import *
import py_wrf_arps
import time
import copy
import scipy
import pandas as pd
%matplotlib inline

In [ ]:
sim = Proj("../../../../", "14_20200515/", ["05"], 
        ["WRF"], 
        tab_test=["30"])
dom = sim.get_dom("05")

In [ ]:
line2 = [[47.24612, -2.563324], 30000.0, 2.5656340004316642] #Ligne perpendiculaire au front à 12h, utilisée dans l'article

itime = "2020-05-17-13"
kw_get = {
    "itime" : itime,
    "vinterp" : {
        "points" : line2,
        "levels" : (300, 301, 10),
    },
    "saved" : {},
}

In [ ]:
from py_wrf_arps import manage_scipy
# W, GWR, GWS = \
# sim.get_data("05", ["W", "GWR", "GWS"], **kw_get)
W, GWR, GWS, GWSM, GWLAM, GWMASK, M2W, GWA, GWAVGW, U, M2U, GWAU, GWAVGU, GWM2R, varU, varW = \
sim.get_data("05", ["W", "GWR", "GWS", "GWSM", "GWLAM", "GWMASK4", "M2W", "GWA", "GWAVGW", "U", "M2U", "GWAU", "GWAVGU", "GWM2R", "GWIM2U", "GWIM2W"], **kw_get)
DX, DY, X, Y = sim.get_data("05", ["DX", "DY", "X", "Y"])

DT = 600
OMEGA = 2*np.pi * GWSM/GWLAM
GWMASK[GWMASK == 0] = np.nan
GWMASK[GWMASK > 0] = 1
GWMASK2 = np.ones(GWMASK.shape)
GWMASK2[GWM2R < 0] = np.nan
GWMASK2[GWM2R > 10000] = np.nan

PHIW = ((GWR+0)%1) * 2*np.pi
Wexp = GWA * np.sin(PHIW) + GWAVGW
PHIU = ((GWR-0.25)%1) * 2*np.pi
Uexp = GWAU * np.sin(PHIU) + GWAVGU


In [ ]:
ix = 80
s = slice(0, None, -1)
params = [{
    "X" : GWM2R*GWMASK2/1000, "Y" : U*GWMASK2, "style" : "-", "kwargs_plt" : {"color" : manage_plot.colors[0],}, 
    "DX_subplots" : 8, "NX_subplots" : 2, "label" : "$U$", "ylim" : [-1, 2.5], "yticks" : [-1, 0, 1, 2],
    "xlabel": "$X_r$ (km)",
    "ylabel": "Velocity (m.s$^{-1}$)",
    "savepath" : "t30/art1/GW_correction_stats_velocity_"+itime, 
    "dpi" : 120,
    "xlim" : [1, 6],
},{ "same" : -1, "Y" : Uexp*GWMASK, "style" : "o-", "same_ax" : True, "label" : None, "label" : "$U_{sine}$",
},{ "same" : -1, "Y" : GWAVGU*GWMASK2, "style" : "--", "label" : "$\widetilde{U}$",
},{ "same" : -1, "Y" : W*GWMASK2, "style" : "-", "label" : "$W$", "kwargs_plt" : {"color" : manage_plot.colors[1],}, 
},{ "same" : -1, "Y" : Wexp*GWMASK, "style" : "o-", "label" : None, "label" : "$W_{sine}$",
},{ "same" : -1, "Y" : GWAVGW*GWMASK2, "style" : "--", "label" : "$\widetilde{W}$",
},{
    "same" : -1, "Y" : M2U*GWMASK2, "style" : "-", "same_fig" : False, "kwargs_plt" : {"color" : manage_plot.colors[0],},
    "ylim" : [0, 0.8], "yticks" : [0, 0.2, 0.4, 0.6, 0.8], "label" : "$\overline{u'^2}$",
    "ylabel": "Velocity variance (m$^2$.s$^{-2}$)",
    "savepath" : "t30/art1/GW_correction_stats_variance_"+itime, 
},{ "same" : -1, "Y" : varU*GWMASK, "style" : "o-", "same_fig" : True, "same_ax" : True, "label" : "$G_U$",
},{ "same" : -1, "Y" : M2W*GWMASK2, "style" : "-", "kwargs_plt" : {"color" : manage_plot.colors[1],}, "label" : "$\overline{w'^2}$",
},{ "same" : -1, "Y" : varW*GWMASK, "style" : "o-", "label" : "$G_W$",
}]
fig = sim.plot_fig(params)